In [ ]:
from pathlib import Path
from tca.text.document_utils import DocumentLoader

document_path = Path(
    "data/accords_entreprise_niveau2/T20A23060010_PROTOCOLE_ANONYME_FIN_CONFLIT_REPRISE_TRAVAIL_202309.docx"
)
document_loaders = DocumentLoader()
document_text = document_loaders.load_text_from_document(document_path)
document_text

In [ ]:
from openai import OpenAI
import pandas as pd

# Configurez votre clé API OpenAI
client = OpenAI()

themes = pd.read_csv("data/theme_list.csv")


def generate_analysis(themes: list[str], document: str) -> list[dict[str, list[str]]]:
    themes_list = "\n".join(f'- "{theme}"' for theme in themes)
    prompt = f"""
Voici l'accord d'entreprise à analyser :
{document}
    """
    # Effectuez l'appel à l'API
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": f"""
Je souhaite analyser un accord d'entreprise d'une entreprise française en fonction des thèmes suivants : {themes_list}"
Retourne les résultats sous le format JSON suivant (sans balisage de bloc de code) :
[
    {{
        "thème": "premier thème qui correspond",
        "extraits": ["bout de texte 1 dans le document qui correspond au thème", "bout de texte 2 dans le document qui correspond au thème"]
    }},
    {{
        "thème": "deuxième thème qui correspond",
        "extraits": ["bout de texte 1 dans le document qui correspond au thème"]
    }}
]
Si un thème ne correspond à aucun extrait dans l'accord, ne l'inclus pas dans la sortie JSON.
Il faut que la correspondance entre le thème et l'extrait trouvé soit forte.
""",
            },
            {"role": "user", "content": prompt},
        ],
        temperature=0,
    )

    return response


result = generate_analysis(themes["description"].to_list(), document_text)
result

In [ ]:
print(result.choices[0].message.content)